# Sprint
## 深層学習スクラッチ 畳み込みニューラルネットワーク1

## 1.このSprintについて

### Sprintの目的
スクラッチを通してCNNの基礎を理解する

### どのように学ぶか
スクラッチで1次元用畳み込みニューラルネットワークを実装した後、学習と検証を行なっていきます。

## 2.1次元の畳み込みニューラルネットワークスクラッチ

**畳み込みニューラルネットワーク（CNN）**のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


このSprintでは1次元の**畳み込み層**を作成し、畳み込みの基礎を理解することを目指します。次のSprintでは2次元畳み込み層とプーリング層を作成することで、一般的に画像に対して利用されるCNNを完成させます。


クラスの名前はScratch1dCNNClassifierとしてください。クラスの構造などは前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。


### 1次元畳み込み層とは
CNNでは画像に対しての2次元畳み込み層が定番ですが、ここでは理解しやすくするためにまずは1次元畳み込み層を実装します。1次元畳み込みは実用上は自然言語や波形データなどの**系列データ**で使われることが多いです。


畳み込みは任意の次元に対して考えることができ、立体データに対しての3次元畳み込みまではフレームワークで一般的に用意されています。


### データセットの用意
検証には引き続きMNISTデータセットを使用します。1次元畳み込みでは全結合のニューラルネットワークと同様に平滑化されたものを入力します。

### 【問題1】
#### チャンネル数を1に限定した1次元畳み込み層クラスの作成
チャンネル数を1に限定した1次元畳み込み層のクラスSimpleConv1dを作成してください。基本構造は前のSprintで作成した全結合層のFCクラスと同じになります。なお、重みの初期化に関するクラスは必要に応じて作り変えてください。Xavierの初期値などを使う点は全結合層と同様です。


ここでは**パディング**は考えず、**ストライド**も1に固定します。また、複数のデータを同時に処理することも考えなくて良く、バッチサイズは1のみに対応してください。この部分の拡張はアドバンス課題とします。


フォワードプロパゲーションの数式は以下のようになります。

$$
a_i = \sum_{s=0}^{F-1}x_{(i+s)}w_s+b
$$

$a_i$ : 出力される配列のi番目の値


$F$ : フィルタのサイズ


$x_{(i+s)}$ : 入力の配列の(i+s)番目の値


$w_s$ : 重みの配列のs番目の値


$b$ : バイアス項


全てスカラーです。


次に更新式です。ここがAdaGradなどに置き換えられる点は全結合層と同様です。

$$
w_s^{\prime} = w_s - \alpha \frac{\partial L}{\partial w_s} \\
b^{\prime} = b - \alpha \frac{\partial L}{\partial b}
$$

$\alpha$ : 学習率


$\frac{\partial L}{\partial w_s}$ : $w_s$に関する損失$L$の勾配


$\frac{\partial L}{\partial b}$ : $b$に関する損失$L$の勾配


勾配$\frac{\partial L}{\partial w_s}$や$\frac{\partial L}{\partial b}$を求めるためのバックプロパゲーションの数式が以下です。

$$
\frac{\partial L}{\partial w_s} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}x_{(i+s)}\\
\frac{\partial L}{\partial b} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}
$$

$\frac{\partial L}{\partial a_i}$ : 勾配の配列のi番目の値


$N_{out}$ : 出力のサイズ


前の層に流す誤差の数式は以下です。

$$
\frac{\partial L}{\partial x_j} = \sum_{s=0}^{F-1} \frac{\partial L}{\partial a_{(j-s)}}w_s
$$

$\frac{\partial L}{\partial x_j}$ : 前の層に流す誤差の配列のj番目の値


ただし、$j−s<0$または$j−s>N_{out}−1$のとき$\frac{\partial L}{\partial a_{(j-s)}}=0$です。


全結合層との大きな違いは、重みが複数の特徴量に対して共有されていることです。この場合は共有されている分の誤差を全て足すことで勾配を求めます。計算グラフ上での分岐はバックプロパゲーションの際に誤差の足し算をすれば良いことになります。

In [1]:
import numpy as np

In [7]:
class SimpleConv1d:
    """
    チャンネル数を1に限定した1次元畳み込み層
    (パディングなし　ストライド1)
    """
    def __init__(self, filter_size, initializer, optimizer):
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.W = initializer.W(filter_size)
        self.B = initializer.B()
        pass

    def forward(self, X):
        """
        フォワード
        """
        self.Z_prev = X

        A = np.zeros((X.shape[0]-self.W.shape[0]+1,))
        for i in range(A.shape[0]):
            A[i] = np.sum(X[i:i+self.W.shape[0]]*self.W) + self.B

        return A

    def backward(self, dA):
        """
        バックワード
        """
        # 更新
        dX = self.optimizer.update(self, dA)

        return dX

In [3]:
class SGD:
    """
    勾配降下法
    """
    def __init__(self, lr):
        self.lr = lr

    def update(self, layer, dA):
        """
        ある層の重みやバイアスの更新
        """
        dB = np.sum(dA)
        dW = np.zeros(layer.W.shape[0])
        for s in range(dW.shape[0]):
            dW[s] = np.dot(layer.Z_prev[s:dA.shape[0]], dA)
        
        dA_padding = np.pad(dA, (layer.W.shape[0]-1, max((0, layer.W.shape[0]-dA.shape[0]))), mode='constant')
        dX = np.zeros(layer.W.shape[0])
        for s in range(dX.shape[0]):
            dX[s] = np.sum(dA_padding[s:s+layer.W.shape[0]]*layer.W[::-1])

        layer.B = layer.B - self.lr*dB
        layer.W = layer.W - self.lr*dW

        return dX

### 【問題2】
#### 1次元畳み込み後の出力サイズの計算
畳み込みを行うと特徴量の数が変化します。どのように変化するかは以下の数式から求められます。パディングやストライドも含めています。この計算を行う関数を作成してください。

$$
N_{out} =  \frac{N_{in}+2P-F}{S} + 1\\
$$

$N_{out}$ : 出力のサイズ（特徴量の数）


$N_{in}$ : 入力のサイズ（特徴量の数）


$P$ : ある方向へのパディングの数


$F$ : フィルタのサイズ


$S$ : ストライドのサイズ

In [4]:
def calc_output_size(n_in, pad_size, filter_size, stride_size):
    n_out = ((n_in+2*pad_size-filter_size) / stride_size) + 1
    return n_out

### 【問題3】
#### 小さな配列での1次元畳み込み層の実験
次に示す小さな配列でフォワードプロパゲーションとバックプロパゲーションが正しく行えているか確認してください。


入力x、重みw、バイアスbを次のようにします。

```python
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])
```

フォワードプロパゲーションをすると出力は次のようになります。

```python
a = np.array([35, 50])
```

次にバックプロパゲーションを考えます。誤差は次のようであったとします。

```python
delta_a = np.array([10, 20])
```

バックプロパゲーションをすると次のような値になります。

``` python
delta_b = np.array([30])
delta_w = np.array([50, 80, 110])
delta_x = np.array([30, 110, 170, 140])
```

In [8]:
# forward
x = np.array([1, 2, 3, 4])
w = np.array([3, 5, 7])
b = np.array([1])

a = np.zeros((x.shape[0]-w.shape[0]+1,))
for i in range(a.shape[0]):
    a[i] = np.sum(x[i:i+w.shape[0]]*w) + b

print(a)
## a = np.array([35, 50])

[35. 50.]


In [12]:
# backward
delta_a = np.array([10, 20])

delta_b = np.sum(delta_a)
delta_w = np.zeros(w.shape[0])
for s in range(delta_w.shape[0]):
    delta_w[s] = np.dot(x[s:s+delta_a.shape[0]], delta_a)

delta_x = np.zeros(x.shape[0])
delta_a_padding = np.pad(delta_a, (w.shape[0]-1, max((0, x.shape[0]-delta_a.shape[0]))), mode='constant')
for s in range(delta_x.shape[0]):
    delta_x[s] = np.sum(delta_a_padding[s:s+w.shape[0]]*w[::-1])

print(delta_b)
## delta_b = np.array([30])
print(delta_w)
## delta_w = np.array([50, 80, 110])
print(delta_x)
## delta_x = np.array([30, 110, 170, 140])

30
[ 50.  80. 110.]
[ 30. 110. 170. 140.]


### 実装上の工夫
畳み込みを実装する場合は、まずはfor文を重ねていく形で構いません。しかし、できるだけ計算は効率化させたいため、以下の式を一度に計算する方法を考えることにします。

$$
a_i = \sum_{s=0}^{F-1}x_{(i+s)}w_s+b
$$

バイアス項は単純な足し算のため、重みの部分を見ます。

$$
\sum_{s=0}^{F-1}x_{(i+s)}w_s
$$

これは、xの一部を取り出した配列とwの配列の内積です。具体的な状況を考えると、以下のようなコードで計算できます。この例では流れを分かりやすくするために、各要素同士でアダマール積を計算してから合計を計算しています。これは結果的に内積と同様です。

```python
x = np.array([1, 2, 3, 4])
w = np.array([3, 5, 7])
a = np.empty((2, 3))
indexes0 = np.array([0, 1, 2]).astype(np.int)
indexes1 = np.array([1, 2, 3]).astype(np.int)
a[0] = x[indexes0]*w # x[indexes0]は([1, 2, 3])である
a[1] = x[indexes1]*w # x[indexes1]は([2, 3, 4])である
a = a.sum(axis=1)
````

ndarrayは配列を使ったインデックス指定ができることを利用した方法です。


また、二次元配列を使えば一次元配列から二次元配列が取り出せます。

```python
x = np.array([1, 2, 3, 4])
indexes = np.array([[0, 1, 2], [1, 2, 3]]).astype(np.int)
print(x[indexes]) # ([[1, 2, 3], [2, 3, 4]])
```

このこととブロードキャストなどをうまく組み合わせることで、一度にまとめて計算することも可能です。


畳み込みの計算方法に正解はないので、自分なりに効率化していってください。


**《参考》**


以下のページのInteger array indexingの部分がこの方法についての記述です。


[Indexing — NumPy v1.17 Manual]()